In [ ]:

import sys
sys.path.insert(0, 'BDT_models')
sys.path.insert(0, 'backend_functions')

import selection_functions as sf

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb


import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT
from ROOT import TH1F, TDirectory, TH1D

import os


import top 
importlib.reload(top)
from top import *


In [ ]:
import uncertainty_functions
from uncertainty_functions import *

In [ ]:
ISRUN3 = False

In [ ]:
if not ISRUN3: 
    standard_dict = detvar_run1_fhc
    intrinsic_dict = intrinsic_detvar_run1_fhc
    
    run = 'run1'
    detvar = detvar_run1_fhc
    
else: 
    standard_dict = detvar_run3_rhc
    intrinsic_dict = intrinsic_detvar_run3_rhc
    
    run = 'run3b'
    detvar = detvar_run3_rhc

In [ ]:


variables = [
    "nu_pdg", "ccnc",
    "true_nu_vtx_x", "true_nu_vtx_y" , "true_nu_vtx_z", 
    "swtrig_pre", "npion", "nproton", "npi0",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "n_tracks_contained", 
    "shr_tkfit_dedx_Y",  
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trkshrhitdist2",
    "n_showers_contained",
    "shr_score", "tksh_angle", 
    "trk_energy", 
    "tksh_distance",
    "shr_energy_tot_cali",  
    "nslice", 
    "contained_fraction", 
    "shrmoliereavg", 
    "ppfx_cv", "weightSplineTimesTune", "NeutrinoEnergy2"
]

In [ ]:
input_path = "/uboone/data/users/kmiller/uBNuMI_CCNp/ntuples/"+run+"/systematics/detvar/"


### load variations

In [ ]:
detvar.keys()

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

In [ ]:
beamon_pot = parameters(ISRUN3)['beamon_pot']
beamon_pot

In [ ]:
#xvar = 'shr_tkfit_dedx_Y'
#bins = [x*0.5 for x in range(15)]

xvar = "reco_nu_vtx_sce_x"
bins = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

q = BDT_LOOSE_CUTS + ' and BDT_score>'+str(parameters(ISRUN3)['bdt_score_cut'])
print("BDT score >", parameters(ISRUN3)['bdt_score_cut'])

In [ ]:
bdt_model = xgb.Booster({'nthread': 4})
bdt_model.load_model(parameters(ISRUN3)['bdt_model'])

useBDT = True

In [ ]:
detvar_dict = {}

In [ ]:
for variation in detvar.keys(): 
    
    print("Plotting", variation)
    
    #standard_input_file = input_path + "standard_overlay/neutrinoselection_filt_run1_overlay_"+variation+".root"
    intrinsic_input_file = input_path + "intrinsic/neutrinoselection_filt_"+run+"_overlay_"+variation+"_intrinsic.root"
    
    #f_standard = uproot.open(standard_input_file)[fold][tree]
    f_intrinsic = uproot.open(intrinsic_input_file)[fold][tree]
    
    uproot_v = [f_intrinsic] #[f_standard, f_intrinsic]
    
    #df_standard = f_standard.pandas.df(variables, flatten=False)
    df_intrinsic = f_intrinsic.pandas.df(variables, flatten=False)
    
    for i, df in enumerate([df_intrinsic]): #([df_standard, df_intrinsic]):
            
        up = uproot_v[i]
        trk_llr_pid_v = up.array('trk_llr_pid_score_v')
        trk_id = up.array('trk_id')-1 
        trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
        df['trkpid'] = trk_llr_pid_v_sel
        df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
        
        df['NeutrinoEnergy2_GeV'] = df['NeutrinoEnergy2']/1000

        df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
        df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
        df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
        df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
            
        # bool for is signal vs is not signal 
        df['is_signal'] = np.where((df.swtrig_pre == 1) 
                            & (df.nu_pdg==12) & (df.ccnc==0) & (df.nproton>0) & (df.npion==0) & (df.npi0==0)
                            & (10 <= df.true_nu_vtx_x) & (df.true_nu_vtx_x <= 246)
                            & (-106 <= df.true_nu_vtx_y) & (df.true_nu_vtx_y <= 106)
                            & (10 <= df.true_nu_vtx_z) & (df.true_nu_vtx_z <= 1026), True, False)
            
        print('is_signal check:', len(df) == len(df.query('is_signal==True')) + len(df.query('is_signal==False')))
        
    # APPLY SW TRIGGER
    #df_standard = df_standard.query('swtrig_pre==1')
    df_intrinsic = df_intrinsic.query('swtrig_pre==1')
    
    # SPLIT DATAFRAME 
    n_rows = len(df_intrinsic) 
    df1 = df_intrinsic.iloc[:round(n_rows/3),:]
    df2 = df_intrinsic.iloc[round(n_rows/3):2*round(n_rows/3),:]
    df3 = df_intrinsic.iloc[2*round(n_rows/3):,:]
    
    print('split check:', len(df_intrinsic) == len(df1) + len(df2) + len(df3))
    
    # POT SCALING
    #df_standard['pot_scale'] = beamon_pot/standard_dict.get(variation)    
    df_intrinsic['pot_scale'] = beamon_pot/intrinsic_dict.get(variation+'_intrinsic')
    
    #df1['pot_scale'] = beamon_pot/((1/3)*intrinsic_dict.get(variation+'_intrinsic'))
    #df2['pot_scale'] = beamon_pot/((1/3)*intrinsic_dict.get(variation+'_intrinsic'))
    #df3['pot_scale'] = beamon_pot/((1/3)*intrinsic_dict.get(variation+'_intrinsic'))
    
    # remove nue CC events 
    #print("# nueCC in AV in standard overlay det. sys. sample = "+str(len(df_standard.query(nueCC_query))))
    #len1 = len(df_standard)

    #idx = df_standard.query(nueCC_query).index
    #df_standard.drop(idx, inplace=True)
    #len2 = len(df_standard) 

    #print("# of nueCC in AV removed = "+str(len1-len2)) # should be same as above
    
    if useBDT: 
            
        print('Using BDT')
            
        #df_bdt_standard = df_standard.copy()
        df_bdt_intrinsic = df_intrinsic.copy()

        # clean datasets 
        for column in training_parameters:
            #df_bdt_standard.loc[(df_bdt_standard[column] < -1.0e37) | (df_bdt_standard[column] > 1.0e37), column] = np.nan
            df_bdt_intrinsic.loc[(df_bdt_intrinsic[column] < -1.0e37) | (df_bdt_intrinsic[column] > 1.0e37), column] = np.nan
    
        # apply BDT model  
        #df_test_standard = xgb.DMatrix(data=df_bdt_standard[training_parameters])
        #preds_standard = bdt_model.predict(df_test_standard)
        #df_bdt_standard['BDT_score'] = preds_standard
        
        df_test_intrinsic = xgb.DMatrix(data=df_bdt_intrinsic[training_parameters])
        preds_intrinsic = bdt_model.predict(df_test_intrinsic)
        df_bdt_intrinsic['BDT_score'] = preds_intrinsic
            
        #df_standard_sel = df_bdt_standard.query(q).copy()
        df_intrinsic_sel = df_bdt_intrinsic.query(q).copy()
        
    else: 
        
        #df_standard_sel = df_standard.query(q).copy()
        df_intrinsic_sel = df_intrinsic.query(q).copy()
        #df1_sel = df1.query(q).copy()
        #df2_sel = df2.query(q).copy()
        #df3_sel = df3.query(q).copy()

    
    #standard_counts = plt.hist(df_standard_sel[xvar], bins, range=[bins[0], bins[-1]], 
    #        weights=df_standard_sel['ppfx_cv']*df_standard_sel['weightSplineTimesTune']*df_standard_sel['pot_scale'])[0]
    #plt.close()
    
    #print("non-nueCC events = ", sum(standard_counts))
    
    intrinsic_counts = plt.hist(df_intrinsic_sel[xvar], bins, range=[bins[0], bins[-1]], 
            weights=df_intrinsic_sel['ppfx_cv']*df_intrinsic_sel['weightSplineTimesTune'])[0] #*df_intrinsic_sel['pot_scale'])[0]
    plt.close()

    
    print("nue CC events = ", sum(intrinsic_counts))

    
    # store counts in a dictionary 
    #detvar_dict[variation] = standard_counts
    detvar_dict[variation+'_intrinsic'] = list(intrinsic_counts)
    #detvar_dict[variation+'_total'] = [a+b for a,b in zip(standard_counts, intrinsic_counts)]

    

In [ ]:
x_err = [ round(abs(bins[x+1]-bins[x])/2, 3) for x in range(len(bins)-1) ]
bin_centers = []
for a in range(len(bins)-1): 
    bin_centers.append(round(bins[a] + (bins[a+1]-bins[a])/2, 3))

In [ ]:
# plt CV 

plt.errorbar(bin_centers, detvar_dict['CV_intrinsic'], xerr=x_err, fmt='none', color='black', linewidth=2, label='CV')
plt.xlabel(xvar, fontsize=14)
plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)
plt.title(variation+' (nue CC events)')
plt.legend()
plt.show()

detvar_dict['CV_intrinsic']

## intrinsic sample

In [ ]:
for variation in detvar: 
    
    if variation=='CV': 
        continue
    
    plt.hist(bin_centers, bins, histtype='step', 
                        range=[bins[0], bins[-1]], weights=detvar_dict[variation+'_intrinsic'], 
                        color='cornflowerblue', linewidth=0.5, label='UV')
    
    plt.errorbar(bin_centers, detvar_dict['CV_intrinsic'], xerr=x_err, fmt='none', color='black', linewidth=2, label='CV')
    plt.xlabel(xvar, fontsize=14)
    plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)
    plt.title(variation+' (nue CC events)')
    plt.legend()
    plt.show()

In [ ]:
detsys_cov_nueCC = {}



for variation in detvar.keys(): 
    
    if variation=='CV': 
        continue
   
    detsys_cov_nueCC[variation+"_to_nueCC"] = calcCov(xvar, bins, detvar_dict['CV_intrinsic'], 
                                              detvar_dict['CV_intrinsic'], 
                                              [detvar_dict[variation+'_intrinsic']])
  

In [ ]:
# TOTAL cov, corr, fractional uncertainty -- nue CC 


cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for i in range(len(bins)-1): 
        for j in range(len(bins)-1):
            
            cov[i][j] = sum([detsys_cov_nueCC[x+"_to_nueCC"]['cov'][i][j] for x in detvar.keys() if x is not 'CV'])
            frac_cov[i][j] = sum([detsys_cov_nueCC[x+"_to_nueCC"]['frac_cov'][i][j] for x in detvar.keys() if x is not 'CV'])

for i in range(len(bins)-1): 
    for j in range(len(bins)-1):
        
        if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
            
nueCC_detsys_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'cor' : cor,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
} 

nueCC_detsys_dict['fractional_uncertainty']

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, nueCC_detsys_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

        
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


plt.xlabel(xvar, fontsize=15)
plt.ylabel(xvar, fontsize=15)

plt.title('Correlation (nueCC)', fontsize=15)

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, nueCC_detsys_dict['frac_cov'], cmap='OrRd', edgecolors='k', vmin=-0.1, vmax=0.1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

        
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


plt.xlabel(xvar, fontsize=15)
plt.ylabel(xvar, fontsize=15)

plt.title('Fractional Covariance (nueCC)', fontsize=15)

plt.show()

In [ ]:
sum(detvar_dict['CV_intrinsic'])

In [ ]:
bincenters = 0.5*(np.array(bins)[1:]+np.array(bins)[:-1])
x_err = [ round(abs(bins[x+1]-bins[x])/2, 3) for x in range(len(bins)-1) ]

plt.figure(figsize=(8, 5))

cv_bdtcut = detvar_dict['CV_intrinsic']
cv_bdtcut_err = [np.sqrt(x)/x for x in cv_bdtcut]

plt.errorbar(bincenters, nueCC_detsys_dict['fractional_uncertainty'], fmt='none', 
             yerr=cv_bdtcut_err, color='blue', alpha=0.7)
plt.hist(bincenters, bins, weights=nueCC_detsys_dict['fractional_uncertainty'], histtype='step', 
         label='After BDT Cut (from 8767.0 GENIE/PPFX-tuned CV evts)', color='blue',alpha=0.7)

plt.hlines(np.average(nueCC_detsys_dict['fractional_uncertainty']), 0, 250, linestyle='--', 
           label='Avg = '+str(round(np.average(nueCC_detsys_dict['fractional_uncertainty']), 3)))

plt.xlim(0, 250)
plt.ylim(0, 0.33)
plt.legend(fontsize=13)

plt.xlabel("Reconstructed X Position [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("FHC Run 1 Det. Sys. ($\\nu_{e}$ CC Intrinsic Sample)", fontsize=15)
#plt.savefig(parameters(ISRUN3)['plots_path']+"RHCRUN3DetSys.pdf", transparent=True, bbox_inches='tight') 

plt.show()

In [ ]:
save_variations = False

In [ ]:
if save_variations: 
    
    import json
    
    with open('variations/FHCDetSysVariations_July8.json', 'w') as f:
        json.dump(detvar_dict, f)


## standard sample

In [ ]:
for variation in detvar: 
    
    if variation=='CV': 
        continue
    
    plt.hist(bin_centers, bins, histtype='step', 
                        range=[bins[0], bins[-1]], weights=detvar_dict[variation], 
                        color='cornflowerblue', linewidth=0.5, label='UV')
    
    plt.errorbar(bin_centers, detvar_dict['CV'], 
                 xerr=x_err, fmt='none', color='black', linewidth=2, label='CV')
    plt.xlabel(xvar, fontsize=14)
    plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)
    plt.title(variation+' (non-nue CC events)')
    plt.legend()
    plt.show()

In [ ]:
detsys_cov_non_nueCC = {}


for variation in detvar.keys(): 
    
    if variation=='CV': 
        continue
    
    # calc covariance for each unisim - nonnueCC 
    detsys_cov_non_nueCC[variation+"_to_nonnueCC"] = calcCov(xvar, bins, detvar_dict['CV'], 
                                              detvar_dict['CV'], # taken from uncertainty.ipynb
    

In [ ]:
#  cov, corr, fractional uncertainty -- non nue CC


cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for i in range(len(bins)-1): 
        for j in range(len(bins)-1):
            
            cov[i][j] = sum([detsys_cov_non_nueCC[x+"_tofull"]['cov'][i][j] for x in detvar.keys() if x is not 'CV'])
            frac_cov[i][j] = sum([detsys_cov_non_nueCC[x+"_tofull"]['frac_cov'][i][j] for x in detvar.keys() if x is not 'CV'])

for i in range(len(bins)-1): 
    for j in range(len(bins)-1):
        
        if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
            
non_nueCC_detsys_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'cor' : cor,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
} 

In [ ]:
non_nueCC_detsys_dict['fractional_uncertainty']

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, non_nueCC_detsys_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

        
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


plt.xlabel(xvar, fontsize=15)
plt.ylabel(xvar, fontsize=15)

plt.title('Correlation (non-nueCC)', fontsize=15)

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, non_nueCC_detsys_dict['frac_cov'], cmap='OrRd', edgecolors='k', vmin=-0.1, vmax=0.1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

        
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


plt.xlabel(xvar, fontsize=15)
plt.ylabel(xvar, fontsize=15)

plt.title('Fractional Covariance (non-nueCC)', fontsize=15)

plt.show()

## comparisons between intrinsic vs. standard overlay

In [ ]:
plt.hist(bin_centers, bins, weights=nueCC_detsys_dict['fractional_uncertainty'], histtype='step', label='nueCC')
plt.hist(bin_centers, bins, weights=non_nueCC_detsys_dict['fractional_uncertainty'], histtype='step', label='non-nueCC')
plt.hist(bin_centers, bins, weights=tot_frac, histtype='step', label='total', color='black', linestyle='--')

#plt.hist(bin_centers, bins, weights=tot_detsys_dict['fractional_uncertainty'], histtype='step', label='together', color='red')

plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.ylim(0, 1.2)
#plt.xlim(0, 7)
plt.ylabel('Fractional Uncertainty', fontsize=14)
plt.title("Detector Uncertainties", fontsize=15)
plt.show()

In [ ]:
np.average(nueCC_detsys_dict['fractional_uncertainty'])

In [ ]:
plt.hist(bin_centers, bins, weights=nueCC_detsys_dict['fractional_uncertainty'], histtype='step', label='nueCC')
plt.hlines(np.average(nueCC_detsys_dict['fractional_uncertainty']), 0, 250, linestyle='--', 
           label='Average = '+str(round(np.average(nueCC_detsys_dict['fractional_uncertainty']), 3)))
plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.ylim(0, 0.65)
plt.xlim(0, bins[-1])
plt.ylabel('Fractional Uncertainty', fontsize=14)
plt.title("Detector Uncertainties", fontsize=15)
plt.show()

In [ ]:
np.average(nueCC_detsys_dict['fractional_uncertainty'])

## OUTDATED

In [ ]:
df_pre = overlay.query(BDT_PRE_QUERY).copy()
df_loose = overlay.query(BDT_LOOSE_CUTS).copy()


In [ ]:
w_pre = list(df_pre['ppfx_cv']*df_pre['weightSplineTimesTune']*df_pre['pot_scale'])
w_loose = list(df_loose['ppfx_cv']*df_loose['weightSplineTimesTune']*df_loose['pot_scale'])

In [ ]:
xvar = 'shr_tkfit_dedx_Y'
xlow = 0
xhigh = 7
bins = [x*0.5 for x in range(15)]
x_label = "dE/dx on the Y plane [MeV/cm]"


plt.hist(df_pre[xvar], bins=bins, range=[bins[0], bins[-1]], weights=w_pre, 
         histtype='step', label="pre")
plt.hist(df_loose[xvar], bins=bins, range=[bins[0], bins[-1]], weights=w_loose, 
         histtype='step', label='loose')
plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.title('LYRayleigh', fontsize=15)
plt.show()

In [ ]:
beamon_pot

In [ ]:
bincenters = 0.5*(np.array(bins)[1:]+np.array(bins)[:-1])
x_err = [ round(abs(bins[x+1]-bins[x])/2, 3) for x in range(len(bins)-1) ]

In [ ]:
# evolution -- total 

plt.figure(figsize=(8, 5))

presel = np.array([0.08919758, 0.03710655, 0.07212393, 0.0740782 , 0.05647801, 0.06390213, 0.03153526, 0.09894273, 0.06314468, 0.08524242])
loosecuts = np.array([0.15597447, 0.13987804, 0.07696078, 0.07240133, 0.10087724, 0.10312506, 0.13383267, 0.05475324, 0.10502673, 0.17370385])
bdt_cut = np.array([0.16095705, 0.29032498, 0.23506311, 0.15685235, 0.33354353, 0.15485916, 0.2284225 , 0.12265416, 0.08450959, 0.21393108])


plt.hist(bincenters, weights=presel, histtype='step', label='After preselection')
plt.hist(bincenters, weights=loosecuts, histtype='step', label='After loose cuts')
plt.hist(bincenters, weights=bdt_cut, histtype='step', label='After BDT cut')

plt.ylim(0, 0.5)
plt.legend(fontsize=13)

plt.xlabel("Reco X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("DetSys Uncertainty Evolution (Total)", fontsize=15)

plt.show()

In [ ]:
bincenters

In [ ]:
# evolution -- nue CC

plt.figure(figsize=(8, 5))

#######################################

presel = np.array([0.06870054, 0.02259937, 0.0497535 , 0.02650766, 0.05251311, 0.01690655, 0.06395052, 0.06882898, 0.07349036, 0.11812196])
cv_presel = np.array([1950.3359375 , 3777.09765625, 3911.71777344, 3808.6484375 , 3780.08691406, 3747.08398438, 3643.8046875 , 3765.2890625 , 3746.96875 , 1781.91210938])
cv_presel_err = [np.sqrt(x)/x for x in cv_presel]

plt.errorbar(bincenters, presel, fmt='none', yerr=cv_presel_err, color='cornflowerblue')
plt.hist(bincenters, bins, weights=presel,  histtype='step', color='cornflowerblue', 
         label='After preselection (33913.0 CV evts, Avg='+str(round(np.average(presel), 3))+')')
#plt.hlines(np.average(presel), 0, 250, linestyle='--', label='Average = '+str(round(np.average(presel), 3)))



#######################################

#plt.figure(figsize=(8, 5))

loosecuts = np.array([0.0749319 , 0.10707654, 0.05782089, 0.04730627, 0.05607244, 0.06158666, 0.09371712, 0.08221844, 0.04345454, 0.08520043])
cv_loosecuts = np.array([ 997.53582764, 1834.1015625 , 1922.6262207 , 1901.54052734, 1795.18994141, 1851.43164062, 1814.76269531, 1974.08789062, 2011.9609375 , 983.98144531])
cv_loosecuts_err = [np.sqrt(x)/x for x in cv_loosecuts]

plt.errorbar(bincenters, loosecuts, fmt='none', yerr=cv_loosecuts_err, color='darkorange')
plt.hist(bincenters, bins, weights=loosecuts,  histtype='step', color='darkorange', 
         label='After loose cuts (17087.2 CV evts, Avg='+str(round(np.average(loosecuts), 3))+')')
#plt.hlines(np.average(loosecuts), 0, 250, linestyle='--', label='Average = '+str(round(np.average(loosecuts), 3)))



#######################################
#plt.figure(figsize=(8, 5))

bdt_cut = np.array([0.11615794, 0.09695565, 0.08512074, 0.10490526, 0.16026189, 0.10672332, 0.11354116, 0.14747448, 0.05915643, 0.10043502])
cv_bdtcut = [552.42785645, 1081.7479248, 1195.08251953, 1152.29296875, 1061.40185547, 1154.8359375, 1089.49169922, 1122.58740234, 1207.57714844, 472.75 ]
cv_bdtcut_err = [np.sqrt(x)/x for x in cv_bdtcut]

plt.errorbar(bincenters, bdt_cut, fmt='none', yerr=cv_bdtcut_err, color='forestgreen')
plt.hist(bincenters, bins, weights=bdt_cut, histtype='step', 
         label='After BDT cut (10090.2 CV evts, Avg='+str(round(np.average(bdt_cut), 3))+')', color='forestgreen')
#plt.hlines(np.average(bdt_cut), 0, 250, linestyle='--', label='Average = '+str(round(np.average(bdt_cut), 3)))

plt.xlim(0, 250)
plt.ylim(0, 0.3)
plt.legend(fontsize=13)

plt.xlabel("Reco X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("DetSys Uncertainty Evolution (nueCC WRT sample)", fontsize=15)

plt.show()

In [ ]:
## half the samples
plt.figure(figsize=(8, 5))

presel = np.array([0.06870054, 0.02259937, 0.0497535 , 0.02650766, 0.05251311, 0.01690655, 0.06395052, 0.06882898, 0.07349036, 0.11812196])
cv_presel = np.array([1950.3359375 , 3777.09765625, 3911.71777344, 3808.6484375 , 3780.08691406, 3747.08398438, 3643.8046875 , 3765.2890625 , 3746.96875 , 1781.91210938])
cv_presel_err = [np.sqrt(x)/x for x in cv_presel]

plt.errorbar(bincenters, presel, fmt='none', yerr=cv_presel_err, color='cornflowerblue',linewidth=2)
plt.hist(bincenters, bins, weights=presel,  histtype='step', color='cornflowerblue', 
         label='Full (33913.0 CV evts, Avg='+str(round(np.average(presel), 3))+')', linewidth=2)


presel_1 = np.array([0.08276135, 0.13338453, 0.08900892, 0.14611306, 0.12927223, 0.07762727, 0.09594148, 0.05464803, 0.12272057, 0.12233051])
cv_presel_1 = np.array([ 986.69923139, 1991.25876022, 1979.06098178, 1850.39915699, 1959.67234071, 1883.90602839, 1812.72918274, 1870.95998444, 1902.79260546, 901.90942413])
cv_presel_1_err = [np.sqrt(x)/x for x in cv_presel_1]

plt.errorbar(bincenters, presel_1, fmt='none', yerr=cv_presel_1_err, color='red')
plt.hist(bincenters, bins, weights=presel_1,  histtype='step', 
         label='Half 1 (17139.4 CV evts, Avg='+str(round(np.average(presel_1), 3))+')', color='red')


presel_2 = np.array([0.13550844, 0.15860746, 0.0508528 , 0.11228756, 0.07343269, 0.0638385 , 0.09993951, 0.13137824, 0.08834187, 0.19772075])
cv_presel_2 = np.array([ 963.6359203 , 1785.84381734, 1932.65090755, 1958.23858006, 1820.39336017, 1863.19111967, 1831.11549528, 1894.37794724, 1844.161984 , 880.00029449])
cv_presel_2_err = [np.sqrt(x)/x for x in cv_presel_2]

plt.errorbar(bincenters, presel_2, fmt='none', yerr=cv_presel_2_err, color='goldenrod')
plt.hist(bincenters, bins, weights=presel_2,  histtype='step', 
         label='Half 2 (16773.6 CV evts, Avg='+str(round(np.average(presel_2), 3))+')', color='goldenrod')


plt.xlim(0, 250)
plt.ylim(0, 0.3)
plt.legend(fontsize=13, loc='upper left')

plt.xlabel("Reco X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("DetSys Uncertainty After Preselection (nueCC WRT sample)", fontsize=15)

plt.show()

In [ ]:
# 1/3 of the samples 

plt.figure(figsize=(8, 5))

loosecuts = np.array([0.0749319 , 0.10707654, 0.05782089, 0.04730627, 0.05607244, 0.06158666, 0.09371712, 0.08221844, 0.04345454, 0.08520043])
cv_loosecuts = np.array([ 997.53582764, 1834.1015625 , 1922.6262207 , 1901.54052734, 1795.18994141, 1851.43164062, 1814.76269531, 1974.08789062, 2011.9609375 , 983.98144531])
cv_loosecuts_err = [np.sqrt(x)/x for x in cv_loosecuts]

plt.errorbar(bincenters, loosecuts, fmt='none', yerr=cv_loosecuts_err, color='darkorange', linewidth=2)
plt.hist(bincenters, bins, weights=loosecuts,  histtype='step', color='darkorange', 
         label='Full (17087.2 CV evts, Avg='+str(round(np.average(loosecuts), 3))+')', linewidth=2)

#######################################

loosecuts_1 = np.array([0.21443809, 0.11683203, 0.25497575, 0.22126091, 0.10338683, 0.1914447 , 0.29447061, 0.10896304, 0.11700523, 0.17420665])
cv_loosecuts_1 = np.array([326.09517148, 621.23741241, 613.16593185, 612.67984031, 625.49892382, 633.92312074, 576.34828676, 661.85773279, 675.02504981, 330.61486563])
cv_loosecuts_1_err = [np.sqrt(x)/x for x in cv_loosecuts_1]

plt.errorbar(bincenters, loosecuts_1, fmt='none', yerr=cv_loosecuts_1_err, color='yellowgreen')
plt.hist(bincenters, bins, weights=loosecuts_1,  histtype='step', color='yellowgreen', 
         label='Third 1 (5676.4 CV evts, Avg='+str(round(np.average(loosecuts_1), 3))+')')

#######################################

loosecuts_2 = np.array([0.18178328, 0.18413507, 0.12833944, 0.10598469, 0.21716196, 0.11239651, 0.18992073, 0.20723299, 0.31169509, 0.30546998])
cv_loosecuts_2 = np.array([330.31648308, 622.28601108, 643.09801325, 621.57186267, 573.27452433, 616.69670582, 597.5434631 , 626.24670479, 616.45876978, 290.32520257])
cv_loosecuts_2_err = [np.sqrt(x)/x for x in cv_loosecuts_2]

plt.errorbar(bincenters, loosecuts_2, fmt='none', yerr=cv_loosecuts_2_err, color='plum')
plt.hist(bincenters, bins, weights=loosecuts_2,  histtype='step', color='plum', 
         label='Third 2 (5537.8 CV evts, Avg='+str(round(np.average(loosecuts_2), 3))+')')


#######################################

loosecuts_3 = np.array([0.19695213, 0.19679284, 0.1427169 , 0.17903722, 0.10891259, 0.17262879, 0.16445115, 0.26874798, 0.20139965, 0.35580272])
cv_loosecuts_3 = np.array([341.12370765, 590.57644886, 666.3615749 , 667.28486967, 596.41434205, 600.79566863, 640.87313415, 685.9799037 , 720.47157948, 363.04109147])
cv_loosecuts_3_err = [np.sqrt(x)/x for x in cv_loosecuts_3]

plt.errorbar(bincenters, loosecuts_3, fmt='none', yerr=cv_loosecuts_3_err, color='peru')
plt.hist(bincenters, bins, weights=loosecuts_3,  histtype='step', color='peru', 
         label='Third 3 (5872.9 CV evts, Avg='+str(round(np.average(loosecuts_3), 3))+')')


#######################################



plt.xlim(0, 250)
plt.ylim(0, 0.5)
plt.legend(fontsize=13, loc='upper left')

plt.xlabel("Reco X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("DetSys Uncertainty After Loose Cuts (nueCC WRT sample)", fontsize=15)

plt.show()

In [ ]:
np.sqrt(997.53582764)/997.53582764

In [ ]:
(np.sqrt(3)/3)

In [ ]:
# evolution -- non nue CC

###########################################

plt.figure(figsize=(8, 5))

presel = np.array([0.18320706, 0.07047145, 0.15389311, 0.13855863, 0.10447263, 0.13108198, 0.07348303, 0.16887075, 0.11046296, 0.17272322])
cv_presel = [170.870224, 320.18408203, 321.68411255, 344.71813965, 329.53295898, 337.1027832, 332.45605469, 359.81982422, 308.86206055, 168.31860352]
cv_presel_err = [np.sqrt(x)/x for x in cv_presel]

plt.errorbar(bincenters, presel, fmt='none', yerr=cv_presel_err, color='cornflowerblue')
plt.hist(bincenters, bins, weights=presel, histtype='step', label='After preselection (2993.5 CV evts, Avg=0.13)', color='cornflowerblue')
#plt.hlines(np.average(presel), 0, 250, linestyle='--', label='Average = '+str(round(np.average(presel), 3)))


###########################################


loosecuts = np.array([0.36642883, 0.40933568, 0.30770769, 0.2274781 , 0.27710819, 0.45294262, 0.37625798, 0.18602785, 0.3058598 , 0.36586612])
cv_loosecuts = np.array([33.15947342, 37.07825851, 35.81521606, 41.98361206, 35.39056396, 29.81181335, 50.02552795, 45.67678833, 43.27896118, 37.53433228])
cv_loosecuts_err = [np.sqrt(x)/x for x in cv_loosecuts]

plt.errorbar(bincenters, loosecuts, fmt='none', yerr=cv_loosecuts_err, color='darkorange')
plt.hist(bincenters, bins, weights=loosecuts, histtype='step', label='After loose cuts (389.8 CV evts, Avg=0.33)', color='darkorange')
#plt.hlines(np.average(loosecuts), 0, 250, linestyle='--', label='Average = '+str(round(np.average(loosecuts), 3)))

###########################################


bdt_cut = np.array([1.08089404, 1.41431277, 1.16890459, 0.53734153, 1.73935168, 0.82604116, 1.87221604, 0.4192925 , 0.61276501, 0. ])
cv_bdtcut = [2.75313497, 7.39452553, 6.95109463, 9.85165405, 4.49909401, 4.84739876, 3.12324142, 6.37028122, 4.52698135, 0. ]
cv_bdtcut_err = [np.sqrt(x)/x for x in cv_bdtcut]

plt.errorbar(bincenters, bdt_cut, fmt='none', yerr=cv_bdtcut_err, color='forestgreen')
plt.hist(bincenters, bins, weights=bdt_cut, histtype='step', label='After BDT cut (50.3 CV evts, Avg=0.97)', color='forestgreen')
#plt.hlines(np.average(bdt_cut), 0, 250, linestyle='--', label='Average = '+str(round(np.average(bdt_cut), 3)))


plt.legend(fontsize=13)

plt.xlim(0, 250)
plt.ylim(0, 3)
plt.xlabel("Reco X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("DetSys Uncertainty Evolution (non-nueCC WRT sample)", fontsize=15)

plt.show()

In [ ]:
# nue CC vs non nue CC


plt.figure(figsize=(8, 5))

presel = np.array([0.18320706, 0.07047145, 0.15389311, 0.13855863, 0.10447263, 0.13108198, 0.07348303, 0.16887075, 0.11046296, 0.17272322])
cv_presel = [170.870224, 320.18408203, 321.68411255, 344.71813965, 329.53295898, 337.1027832, 332.45605469, 359.81982422, 308.86206055, 168.31860352]
cv_presel_err = [np.sqrt(x)/x for x in cv_presel]

plt.errorbar(bincenters, presel, fmt='none', yerr=cv_presel_err, color='cornflowerblue')
plt.hist(bincenters, bins, weights=presel, histtype='step', label='non-nueCC after preselection (2993.5 CV evts, Avg=0.13)', color='cornflowerblue')
#plt.hlines(np.average(presel), 0, 250, linestyle='--', label='Average = '+str(round(np.average(presel), 3)))


presel = np.array([0.06870054, 0.02259937, 0.0497535 , 0.02650766, 0.05251311, 0.01690655, 0.06395052, 0.06882898, 0.07349036, 0.11812196])
cv_presel = np.array([1950.3359375 , 3777.09765625, 3911.71777344, 3808.6484375 , 3780.08691406, 3747.08398438, 3643.8046875 , 3765.2890625 , 3746.96875 , 1781.91210938])
cv_presel_err = [np.sqrt(x)/x for x in cv_presel]

plt.errorbar(bincenters, presel, fmt='none', yerr=cv_presel_err, color='darkblue')
plt.hist(bincenters, bins, weights=presel,  histtype='step', color='darkblue', 
         label='nueCC after preselection (33913.0 CV evts, Avg='+str(round(np.average(presel), 3))+')')
#plt.hlines(np.average(presel), 0, 250, linestyle='--', label='Average = '+str(round(np.average(presel), 3)))



plt.legend(fontsize=13)

plt.xlim(0, 250)
plt.ylim(0, .4)
plt.xlabel("Reco X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("DetSys Uncertainty Evolution (fractional WRT sample)", fontsize=15)

plt.show()


In [ ]:
cv_df_standard['pot_scale'] = beamon_pot/standard_dict.get('CV')    
cv_df_intrinsic['pot_scale'] = beamon_pot/intrinsic_dict.get('CV_intrinsic')

In [ ]:
# remove nue CC events 
print("# nueCC in AV in standard overlay det. sys. sample = "+str(len(cv_df_standard.query(nueCC_query))))
len1 = len(cv_df_standard)
    
idx = cv_df_standard.query(nueCC_query).index
cv_df_standard.drop(idx, inplace=True)
len2 = len(cv_df_standard) 
    
print("# of nueCC in AV removed = "+str(len1-len2)) # should be same as above
        
cv_overlay = pd.concat([cv_df_standard,cv_df_intrinsic], ignore_index=True)

In [ ]:
cv_df_pre = cv_overlay.query(BDT_PRE_QUERY).copy()
cv_df_loose = cv_overlay.query(BDT_LOOSE_CUTS).copy()


In [ ]:
cv_w_pre = list(cv_df_pre['ppfx_cv']*cv_df_pre['weightSplineTimesTune']*cv_df_pre['pot_scale'])
cv_w_loose = list(cv_df_loose['ppfx_cv']*cv_df_loose['weightSplineTimesTune']*cv_df_loose['pot_scale'])

In [ ]:

plt.hist(cv_df_pre[xvar], bins=bins, range=[bins[0], bins[-1]], weights=cv_w_pre, 
         histtype='step', label="pre")
plt.hist(cv_df_loose[xvar], bins=bins, range=[bins[0], bins[-1]], weights=cv_w_loose, 
         histtype='step', label='loose')
plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.title('CV', fontsize=15)
plt.show()

In [ ]:
plt.hist(df_loose[xvar], bins=bins, range=[bins[0], bins[-1]], weights=w_loose, 
         histtype='step', label="LYRayleigh")

plt.hist(cv_df_loose[xvar], bins=bins, range=[bins[0], bins[-1]], weights=cv_w_loose, 
         histtype='step', label='CV')
plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)

plt.show()

In [ ]:
beamon_pot

In [ ]:

cv_overlay_counts = plt.hist(cv_overlay[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=cv_overlay['ppfx_cv']*cv_overlay['weightSplineTimesTune']*cv_overlay['pot_scale'], 
         histtype='step', label="CV full")[0]
overlay_counts = plt.hist(overlay[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=overlay['ppfx_cv']*overlay['weightSplineTimesTune']*overlay['pot_scale'], 
         histtype='step', label='LYRayleigh full')[0]

plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)
plt.title('Before any selection', fontsize=15)
plt.show()

In [ ]:
sum(overlay_counts)

In [ ]:


cv_overlay_counts = plt.hist(cv_overlay.query(BDT_LOOSE_CUTS)[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=cv_overlay.query(BDT_LOOSE_CUTS)['ppfx_cv']*cv_overlay.query(BDT_LOOSE_CUTS)['weightSplineTimesTune']*cv_overlay.query(BDT_LOOSE_CUTS)['pot_scale'], 
         histtype='step', label="CV full")[0]
overlay_counts = plt.hist(overlay.query(BDT_LOOSE_CUTS)[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=overlay.query(BDT_LOOSE_CUTS)['ppfx_cv']*overlay.query(BDT_LOOSE_CUTS)['weightSplineTimesTune']*overlay.query(BDT_LOOSE_CUTS)['pot_scale'], 
         histtype='step', label='LYRayleigh full')[0]

plt.legend(fontsize=14, loc='upper left')
plt.xlabel(xvar, fontsize=14)
plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)
plt.title('After loose cuts', fontsize=15)
plt.show()

In [ ]:
# compare only the intrinsic samples 

cv_intrinsic_counts = plt.hist(cv_df_intrinsic[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=cv_df_intrinsic['ppfx_cv']*cv_df_intrinsic['weightSplineTimesTune']*cv_df_intrinsic['pot_scale'], 
         histtype='step', label="CV intrinsic")[0]
intrinsic_counts = plt.hist(df_intrinsic[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=df_intrinsic['ppfx_cv']*df_intrinsic['weightSplineTimesTune']*df_intrinsic['pot_scale'], 
         histtype='step', label='LYRayleigh intrinsic')[0]

plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.ylim(0, 1000)
plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)
plt.title('Before any selection', fontsize=15)
plt.show()

In [ ]:
print('cv intrinsic counts', sum(cv_intrinsic_counts))
print('LYRayleigh intrinsic counts', sum(intrinsic_counts))

In [ ]:
# compare only the intrinsic samples 

cv_counts = plt.hist(cv_df_standard[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=cv_df_standard['ppfx_cv']*cv_df_standard['weightSplineTimesTune']*cv_df_standard['pot_scale'], 
         histtype='step', label="CV standard (non-nueCC)")[0]
counts = plt.hist(df_standard[xvar], #bins=bins, range=[bins[0], bins[-1]], 
         weights=df_standard['ppfx_cv']*df_standard['weightSplineTimesTune']*df_standard['pot_scale'], 
         histtype='step', label='LYRayleigh standard (non-nueCC)')[0]

plt.legend(fontsize=14)
plt.xlabel(xvar, fontsize=14)
plt.ylabel('$\\nu$ / '+str(beamon_pot)+' POT', fontsize=14)
plt.title('Before any selection', fontsize=15)
#plt.ylim(0, 1300)
plt.show()

In [ ]:
print('cv  counts', sum(cv_counts))
print('LYRayleigh  counts', sum(counts))

In [ ]:
cv_counts